In [ ]:
pip install geoopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 4.7 MB/s eta 0:00:00


In [8]:
import networkx as nx
from itertools import combinations
from joblib import Parallel, delayed
from tqdm import tqdm

def compute_hyperbolicity(G, sample_size=None, n_jobs=-1):
    """
    Compute the hyperbolicity of a graph G.
    Hyperbolicity is calculated based on the Gromov's four-point condition.

    Parameters:
        G (networkx.Graph): The input graph (can be disconnected).
        sample_size (int, optional): Number of random quadruples to sample. If None, evaluates all quadruples.
        n_jobs (int, optional): Number of parallel jobs. Defaults to -1 (use all available cores).

    Returns:
        float: The hyperbolicity of the graph.
    """
    import random
    random.seed(88)

    # Initialize hyperbolicity
    delta = 0

    # Identify connected components
    components = list(nx.connected_components(G))

    # Get all nodes
    nodes = list(G.nodes())

    # Generate quadruples
    if sample_size is None:
        quadruples = list(combinations(nodes, 4))
    else:
        quadruples = [tuple(random.sample(nodes, 4)) for _ in range(sample_size)]

    def process_quadruple(quad):
        u, v, x, y = quad

        # Check if all nodes belong to the same connected component
        component = next((comp for comp in components if {u, v, x, y}.issubset(comp)), None)
        if component is None:
            return 0

        # Compute shortest paths dynamically
        try:
            subgraph = G.subgraph(component)
            d_uv = nx.shortest_path_length(subgraph, source=u, target=v)
            d_ux = nx.shortest_path_length(subgraph, source=u, target=x)
            d_uy = nx.shortest_path_length(subgraph, source=u, target=y)
            d_vx = nx.shortest_path_length(subgraph, source=v, target=x)
            d_vy = nx.shortest_path_length(subgraph, source=v, target=y)
            d_xy = nx.shortest_path_length(subgraph, source=x, target=y)
        except nx.NetworkXNoPath:
            return 0

        # Compute the three sums
        s1 = d_uv + d_xy
        s2 = d_ux + d_vy
        s3 = d_uy + d_vx

        # Compute the maximum of these sums
        max_s = max(s1, s2, s3)

        # Compute the hyperbolicity for this quadruple
        delta_quad = (max_s - min(s1, s2, s3)) / 2

        return delta_quad

    # Process quadruples in parallel
    results = Parallel(n_jobs=n_jobs)(delayed(process_quadruple)(quad) for quad in tqdm(quadruples, desc="Processing quadruples"))

    # Compute final hyperbolicity
    delta = max(results)

    return delta

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter
import os
from tqdm import tqdm
import pandas as pd

##############################################
# Data Loading and Preprocessing
##############################################

# def load_fb15k_data(train_path, valid_path, test_path):
#     def read_triples(path, ent2id, rel2id):
#         triples = []
#         df = pd.read_csv(path, header=None, names=["head", "relation", "tail"])
#         for _, row in df.iterrows():
#             h, r, t = row["head"], row["relation"], row["tail"]
#             if h not in ent2id:
#                 ent2id[h] = len(ent2id)
#             if t not in ent2id:
#                 ent2id[t] = len(ent2id)
#             if r not in rel2id:
#                 rel2id[r] = len(rel2id)
#             triples.append((ent2id[h], rel2id[r], ent2id[t]))
#         return triples

#     ent2id = {}
#     rel2id = {}
#     train_triples = read_triples(train_path, ent2id, rel2id)
#     valid_triples = read_triples(valid_path, ent2id, rel2id)
#     test_triples = read_triples(test_path, ent2id, rel2id)

#     G = nx.Graph()
#     G.add_nodes_from(range(len(ent2id)))
#     for h, r, t in train_triples:
#         G.add_edge(h, t)

#     return ent2id, rel2id, G, train_triples, valid_triples, test_triples


def load_fb15k_data(train_path, valid_path, test_path):
    def read_triples(path, ent2id, rel2id):
        triples = []
        with open(path, 'r') as f:
            for line in f:
                h, r, t = line.strip().split('\t')
                if h not in ent2id:
                    ent2id[h] = len(ent2id)
                if t not in ent2id:
                    ent2id[t] = len(ent2id)
                if r not in rel2id:
                    rel2id[r] = len(rel2id)
                triples.append((ent2id[h], rel2id[r], ent2id[t]))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(h, t)

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples

# Define file paths
# train_path = 'Wordnet_train.csv'
# valid_path = 'Wordnet_valid.csv'
# test_path = 'Wordnet_test.csv'


train_path = 'FB15k_train.txt'
valid_path = 'FB15k_valid.txt'
test_path = 'FB15k_test.txt'


# Load data
ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_fb15k_data(train_path, valid_path, test_path)

num_entities = len(ent2id)
num_relations = len(rel2id)

print(f"Number of Entities: {num_entities}")
print(f"Number of Relations: {num_relations}")
print(f"Number of Training Triples: {len(train_triples)}")
print(f"Number of Validation Triples: {len(valid_triples)}")
print(f"Number of Test Triples: {len(test_triples)}")

from scipy.sparse import csr_matrix, diags
import numpy as np

# Convert adjacency matrix to a sparse matrix
adj = nx.to_scipy_sparse_array(G, nodelist=range(num_entities)).tocsc()

# Add self-loops
adj += diags([1e-5] * num_entities)

# Compute degree
degrees = np.array(adj.sum(axis=1)).flatten()

# Compute D^-0.5 (inverse square root of degree matrix)
inv_sqrt_deg = np.power(degrees, -0.5)
inv_sqrt_deg[np.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = diags(inv_sqrt_deg)

# Normalize adjacency matrix: D^-0.5 * A * D^-0.5
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt

# Convert back to a PyTorch tensor
# To save memory, keep it on CPU if possible or use sparse tensors
norm_adj = torch.tensor(norm_adj.toarray(), dtype=torch.float32)  # If GPU is available, use .to(device)

# Node Features: Use lower-dimensional learnable embeddings instead of one-hot
embedding_dim = 128  # Reduced dimensionality
# Initialize features as learnable parameters
features = nn.Parameter(torch.randn(num_entities, embedding_dim), requires_grad=True)  # Initialize on CPU

# Move tensors to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

print(f"Using device: {device}")

print("Hyperbolicity:", compute_hyperbolicity(G, sample_size=100)) # INITIAL CALCULATION FOR 1.5 DELTA WAS DONE WITH A SAMPLE SIZE OF 20000 BUT THIS SMALLER SAMPLE RUN IS DONE TO RUN IN A SHORT TIME

Number of Entities: 14541
Number of Relations: 237
Number of Training Triples: 272115
Number of Validation Triples: 17535
Number of Test Triples: 20466
Using device: cuda





Processing quadruples:   0%|          | 0/100 [00:00<?, ?it/s]


Processing quadruples:   1%|          | 1/100 [00:00<00:11,  8.90it/s]


Processing quadruples:   4%|▍         | 4/100 [00:01<00:41,  2.29it/s]


Processing quadruples:   6%|▌         | 6/100 [00:02<00:31,  2.96it/s]


Processing quadruples:   7%|▋         | 7/100 [00:02<00:27,  3.39it/s]


Processing quadruples:   8%|▊         | 8/100 [00:02<00:28,  3.21it/s]


Processing quadruples:  10%|█         | 10/100 [00:03<00:24,  3.63it/s]


Processing quadruples:  12%|█▏        | 12/100 [00:03<00:22,  3.96it/s]


Processing quadruples:  13%|█▎        | 13/100 [00:03<00:20,  4.19it/s]


Processing quadruples:  14%|█▍        | 14/100 [00:03<00:20,  4.19it/s]


Processing quadruples:  16%|█▌        | 16/100 [00:04<00:19,  4.31it/s]


Processing quadruples:  18%|█▊        | 18/100 [00:04<00:18,  4.46it/s]


Processing quadruples:  20%|██        | 20/100 [00:05<00:18,  4.38it/s]


Processing quadruples:  22%|██▏       | 22/100 [0

Hyperbolicity: 1.0


INITIAL CALCULATION FOR 1.5 DELTA WAS DONE WITH A SAMPLE SIZE OF 20000 BUT THIS SMALLER SAMPLE RUN IS DONE TO RUN IN A SHORT TIME